# Initiation of Open AI

In [ ]:
!pip install openai
!pip install --upgrade ir_datasets
import ir_datasets
import openai
import os
import json
import pandas as pd

# # define your OpenAI API key here
openai.api_key = ""

## Paths & Parameters

In [11]:
from pandas.core.internals.array_manager import external_values


model = "gpt-3.5-turbo" # OTHER OPTION is text-davinci-003.
query_prompt_base = """Please identify the patient's medical condition and current treatments, including any alternative names, abbreviations, or synonyms for these terms, as well as any additional criteria that may be important for identifying clinical trials of interest.
Respond with a comma-separated list of keywords that will be used for search. Do not elaborate or explain.
Patient's medical note:
"""

# Queries
dataset = ir_datasets.load("clinicaltrials/2021/trec-ct-2022")
queries = pd.DataFrame(dataset.queries_iter())
del dataset
display(queries)

,query_id,text
0,1,\nA 19-year-old male came to clinic with some ...
1,2,\nA 32-year-old woman comes to the hospital wi...
2,3,\nA 51-year-old man comes to the office compla...
3,4,\nA 66-year-old woman comes to the office due ...
4,5,\nA 23-year-old man comes to the emergency dep...
5,6,\nA 61-year-old man comes to the clinic due to...
6,7,\nA 3-year-old girl is brought to the clinic b...
7,8,\nA 7-month-old boy is brought to emergency by...
8,9,\nA 67-year-old woman comes to the clinic due ...
9,10,\nA 19-year-old girl comes to the clinic due t...


# Information Extraction mechanism with chat gpt3.

Top p and temperature parameters both control the randomness of the model. OpenAI documentation recommends using either one parameter or the other and setting the unused parameter to the neutral case, i.e. 1.0.

Because gpt-3.5-turbo performs at a similar capability to text-davinci-003 but at 10% the price per token, we recommend gpt-3.5-turbo for most use cases.




In [12]:
def create_prompt(query_text):
  prompt=[
   {"role": "system", "content": "You are a helpful medical assistant."} #"You are a helpful medical assistant."
   ,{"role": "user", "content": '\'' + query_prompt_base + ' ' + query_text + '\''}]
  return prompt

def generate_response(model,query_text):
  if model == "text-davinci-003" :
    response = openai.Completion.create(
    prompt=create_prompt(query_text),
    engine=model,
    temperature=0,  # or 0.7
    max_tokens=256,
    top_p=1.0,
    frequency_penalty=2,
    presence_penalty=1.0) # or 1.5

    return response.choices[0].text.replace('\n',', ').replace('Keywords:','')

  elif model == "gpt-3.5-turbo":
    try:
      response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          temperature=0, # or 0.7,
          max_tokens=256,
          top_p=1.0,
          frequency_penalty=1.2,
          presence_penalty=1, #1.5
          messages=create_prompt(query_text))

      answer = response['choices'][0]['message']['content'].replace('\n',', ').replace('Keywords:','')
      # answer = strip_punctuation(answer)
      display(answer)
      return answer

    except openai.error.RateLimitError:
      print('API rate limit exceeded. Waiting 5 seconds before retrying...')
      time.sleep(5)
      return generate_response(model,query_text)

  else:
    print('Invalid model selection.')


queries['prompt_ans'] = queries['text'].apply(lambda x: generate_response('gpt-3.5-turbo',x))

display(queries)

queries.to_excel('/content/queries_2022.xlsx',index=False)

"""hypogonadism, delayed puberty, low testosterone, GnRH deficiency
vaginal spotting, menstrual period, menses, UTIs, appendectomy, male partners, barrier contraceptives, vital signs, serum β-hCG level, pelvic ultrasound
hemochromatosis, iron overload, C282Y mutation, fatigue, lack of libido
joint pain, hands, morning stiffness, Tylenol, hypertension, hypercholesteremia, firm nodules, distal interphalangeal joints, knees, bone friction,, narrowing of joint space,, subchondral bone sclerosis,, osteophyte formation"""

'hypogonadism, delayed puberty, low testosterone, GnRH deficiency'

'vaginal spotting, menstrual period, menses, UTIs, appendectomy, male partners, barrier contraceptives, vital signs, serum β-hCG level, pelvic ultrasound'

'hemochromatosis, iron overload, C282Y mutation, fatigue, lack of libido'

'joint pain, hands, morning stiffness, Tylenol, hypertension, hypercholesteremia, firm nodules, distal interphalangeal joints, knees, bone friction,, narrowing of joint space,, subchondral bone sclerosis,, osteophyte formation'

KeyboardInterrupt: ignored